<a href="https://colab.research.google.com/github/InowaR/colab/blob/main/polinomial_adam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [278]:
import numpy as np
from tabulate import tabulate

# Конфигурация
np.random.seed(42)  # Для воспроизводимости результатов
LEARNING_RATE = 0.01
EPOCHS = 5000
BETA1 = 0.9
BETA2 = 0.999
EPSILON = 1e-8
LOG_INTERVAL = 500

# Данные: стороны прямоугольника (a, b) и их площади (a*b)
X = np.array([
    [1, 1], [2, 2], [3, 3],  # Квадраты
    [1, 2], [2, 3], [3, 4],  # Прямоугольники 1
    [1, 3], [2, 4], [3, 5]   # Прямоугольники 2
])

# Целевые значения (площади)
y = np.array([a*b for a, b in X])

def create_poly_features(X):
    """Создает полиномиальные признаки 2-й степени."""
    a, b = X[:, 0], X[:, 1]
    return np.column_stack([
        np.ones(len(X)),  # Свободный член
        a, b,             # Линейные члены
        a*b,              # Взаимодействие
        a**2, b**2        # Квадраты
    ])

# Создаем матрицу признаков
X_poly = create_poly_features(X)
feature_names = ['1', 'a', 'b', 'a*b', 'a²', 'b²']
n_features = X_poly.shape[1]

# Инициализация весов и моментов Adam
weights = np.random.uniform(-0.1, 0.1, size=n_features)
m = np.zeros(n_features)
v = np.zeros(n_features)

# Обучение с Adam optimizer
for epoch in range(1, EPOCHS + 1):
    # Прямое распространение и вычисление ошибки
    errors = X_poly @ weights - y

    # Градиент
    gradient = X_poly.T @ errors / len(X)

    # Обновление моментов Adam
    m = BETA1 * m + (1 - BETA1) * gradient
    v = BETA2 * v + (1 - BETA2) * gradient**2

    # Коррекция смещения
    m_hat = m / (1 - BETA1**epoch)
    v_hat = v / (1 - BETA2**epoch)

    # Обновление весов
    weights -= LEARNING_RATE * m_hat / (np.sqrt(v_hat) + EPSILON)

    # Логирование
    if epoch % LOG_INTERVAL == 0:
        loss = np.mean(errors**2)
        print(f"Эпоха {epoch:4d}, Loss: {loss:.6f}")

# Вывод результатов обучения
print("\nОбученные веса:")
print(tabulate(zip(feature_names, weights),
              headers=["Признак", "Вес"],
              tablefmt="grid",
              floatfmt=".6f"))

# Проверка на обучающей выборке
train_results = np.column_stack([
    [f"{x}" for x in X],
    y,
    np.round(X_poly @ weights, 2),
    np.round(np.abs(X_poly @ weights - y), 2)
])

print("\nПроверка на обучающих данных:")
print(tabulate(train_results,
              headers=["Стороны", "Истинная площадь", "Предсказание", "Ошибка"],
              tablefmt="grid"))

# Функция для предсказания
def predict_area(rectangles, weights):
    """Предсказывает площадь для массива прямоугольников."""
    a = rectangles[:, 0]
    b = rectangles[:, 1]
    features = np.column_stack([
        np.ones(len(rectangles)), a, b, a*b, a**2, b**2
    ])
    return features @ weights

# Тестирование на новых данных
test_rects = np.array([
    [2.5, 2.5],  # 6.25
    [1.5, 3.5],  # 5.25
    [4.0, 5.0]   # 20.0
])

predictions = predict_area(test_rects, weights)
true_areas = test_rects[:, 0] * test_rects[:, 1]

print("\nТестирование на новых данных:")
test_results = np.column_stack([
    [f"{rect}" for rect in test_rects],
    true_areas,
    np.round(predictions, 2),
    np.round(np.abs(predictions - true_areas), 2)
])

print(tabulate(test_results,
              headers=["Стороны", "Ожидаемая площадь", "Предсказание", "Ошибка"],
              tablefmt="grid"))

Эпоха  500, Loss: 0.045955
Эпоха 1000, Loss: 0.023494
Эпоха 1500, Loss: 0.019860
Эпоха 2000, Loss: 0.016004
Эпоха 2500, Loss: 0.012333
Эпоха 3000, Loss: 0.009168
Эпоха 3500, Loss: 0.006655
Эпоха 4000, Loss: 0.004753
Эпоха 4500, Loss: 0.003312
Эпоха 5000, Loss: 0.002192

Обученные веса:
+-----------+-----------+
| Признак   |       Вес |
+===========+===========+
| 1         |  0.002104 |
+-----------+-----------+
| a         |  0.195875 |
+-----------+-----------+
| b         | -0.174289 |
+-----------+-----------+
| a*b       |  0.792732 |
+-----------+-----------+
| a²        |  0.107479 |
+-----------+-----------+
| b²        |  0.097666 |
+-----------+-----------+

Проверка на обучающих данных:
+-----------+--------------------+----------------+----------+
| Стороны   |   Истинная площадь |   Предсказание |   Ошибка |
+===========+====================+================+==========+
| [1 1]     |                  1 |           1.02 |     0.02 |
+-----------+--------------------+------

In [279]:
import numpy as np
from tabulate import tabulate

# Конфигурация
np.random.seed(42)  # Для воспроизводимости результатов
LEARNING_RATE = 0.01
EPOCHS = 5000
BETA1 = 0.9
BETA2 = 0.999
EPSILON = 1e-8
LOG_INTERVAL = 500

# Данные: стороны прямоугольника (a, b) и их периметры (2*(a + b))
X = np.array([
    [1, 1], [2, 2], [3, 3],  # Квадраты
    [1, 2], [2, 3], [3, 4],  # Прямоугольники 1
    [1, 3], [2, 4], [3, 5]   # Прямоугольники 2
])

# Целевые значения (периметры)
y = np.array([2*(a + b) for a, b in X])

def create_poly_features(X):
    """Создает полиномиальные признаки, включая явную формулу периметра."""
    a, b = X[:, 0], X[:, 1]
    return np.column_stack([
        np.ones(len(X)),      # Свободный член
        a, b,                 # Линейные члены
        2*(a + b),           # Явная формула периметра
        a*b,                  # Взаимодействие
        a**2, b**2,           # Квадраты
        (a + b)**2            # Квадрат суммы сторон
    ])

# Создаем матрицу признаков
X_poly = create_poly_features(X)
feature_names = ['1', 'a', 'b', '2(a+b)', 'a*b', 'a²', 'b²', '(a+b)²']
n_features = X_poly.shape[1]

# Инициализация весов и моментов Adam
weights = np.random.uniform(-0.1, 0.1, size=n_features)
m = np.zeros(n_features)
v = np.zeros(n_features)

# Обучение с Adam optimizer
for epoch in range(1, EPOCHS + 1):
    # Прямое распространение и вычисление ошибки
    errors = X_poly @ weights - y

    # Градиент
    gradient = X_poly.T @ errors / len(X)

    # Обновление моментов Adam
    m = BETA1 * m + (1 - BETA1) * gradient
    v = BETA2 * v + (1 - BETA2) * gradient**2

    # Коррекция смещения
    m_hat = m / (1 - BETA1**epoch)
    v_hat = v / (1 - BETA2**epoch)

    # Обновление весов
    weights -= LEARNING_RATE * m_hat / (np.sqrt(v_hat) + EPSILON)

    # Логирование
    if epoch % LOG_INTERVAL == 0:
        loss = np.mean(errors**2)
        print(f"Эпоха {epoch:4d}, Loss: {loss:.6f}")

# Вывод результатов обучения
print("\nОбученные веса:")
print(tabulate(zip(feature_names, weights),
              headers=["Признак", "Вес"],
              tablefmt="grid",
              floatfmt=".6f"))

# Проверка на обучающей выборке
train_results = np.column_stack([
    [f"{x}" for x in X],
    y,
    np.round(X_poly @ weights, 2),
    np.round(np.abs(X_poly @ weights - y), 2)
])

print("\nПроверка на обучающих данных:")
print(tabulate(train_results,
              headers=["Стороны", "Истинный периметр", "Предсказание", "Ошибка"],
              tablefmt="grid"))

# Функция для предсказания
def predict_perimeter(rectangles, weights):
    """Предсказывает периметр для массива прямоугольников."""
    a = rectangles[:, 0]
    b = rectangles[:, 1]
    features = np.column_stack([
        np.ones(len(rectangles)),
        a, b,
        2*(a + b),
        a*b,
        a**2, b**2,
        (a + b)**2
    ])
    return features @ weights

# Тестирование на новых данных
test_rects = np.array([
    [2.5, 2.5],  # Периметр = 10.0
    [1.5, 3.5],  # Периметр = 10.0
    [4.0, 5.0]   # Периметр = 18.0
])

predictions = predict_perimeter(test_rects, weights)
true_perimeters = np.array([2*(a + b) for a, b in test_rects])

print("\nТестирование на новых данных:")
test_results = np.column_stack([
    [f"{rect}" for rect in test_rects],
    true_perimeters,
    np.round(predictions, 2),
    np.round(np.abs(predictions - true_perimeters), 2)
])

print(tabulate(test_results,
              headers=["Стороны", "Ожидаемый периметр", "Предсказание", "Ошибка"],
              tablefmt="grid"))

Эпоха  500, Loss: 0.021428
Эпоха 1000, Loss: 0.015082
Эпоха 1500, Loss: 0.010720
Эпоха 2000, Loss: 0.006786
Эпоха 2500, Loss: 0.003811
Эпоха 3000, Loss: 0.001926
Эпоха 3500, Loss: 0.000918
Эпоха 4000, Loss: 0.000441
Эпоха 4500, Loss: 0.000213
Эпоха 5000, Loss: 0.000095

Обученные веса:
+-----------+-----------+
| Признак   |       Вес |
+===========+===========+
| 1         |  0.062988 |
+-----------+-----------+
| a         |  0.629785 |
+-----------+-----------+
| b         |  0.723743 |
+-----------+-----------+
| 2(a+b)    |  0.642153 |
+-----------+-----------+
| a*b       | -0.125095 |
+-----------+-----------+
| a²        | -0.038620 |
+-----------+-----------+
| b²        | -0.061654 |
+-----------+-----------+
| (a+b)²    |  0.061283 |
+-----------+-----------+

Проверка на обучающих данных:
+-----------+---------------------+----------------+----------+
| Стороны   |   Истинный периметр |   Предсказание |   Ошибка |
+===========+=====================+================+========

In [280]:
import numpy as np
from tabulate import tabulate

# Конфигурация
np.random.seed(42)  # Для воспроизводимости результатов
LEARNING_RATE = 0.01
EPOCHS = 5000
BETA1 = 0.9
BETA2 = 0.999
EPSILON = 1e-8
LOG_INTERVAL = 500

# Генерация данных: диаметр (d), высота (h) и объем цилиндра (π*(d/2)²*h)
def generate_cylinder_data(n_samples=20):
    np.random.seed(42)
    d = np.random.uniform(1, 10, size=n_samples)  # Диаметр от 1 до 10
    h = np.random.uniform(1, 10, size=n_samples)  # Высота от 1 до 10
    X = np.column_stack([d, h])
    y = np.pi * (d/2)**2 * h  # Объем цилиндра
    return X, y

X, y = generate_cylinder_data()

def create_poly_features(X):
    """Создает полиномиальные признаки, включая явную формулу объема цилиндра."""
    d, h = X[:, 0], X[:, 1]
    r = d/2  # Радиус
    return np.column_stack([
        np.ones(len(X)),      # Свободный член
        d, h,                 # Линейные члены
        np.pi * (r**2) * h,  # Явная формула объема V=πr²h
        d*h,                  # Взаимодействие
        d**2, h**2,           # Квадраты
        (d + h)**2,           # Квадрат суммы
        r**2 * h              # r²h (часть формулы объема)
    ])

# Создаем матрицу признаков
X_poly = create_poly_features(X)
feature_names = ['1', 'd', 'h', 'π(d/2)²h', 'd*h', 'd²', 'h²', '(d+h)²', '(d/2)²h']
n_features = X_poly.shape[1]

# Инициализация весов и моментов Adam
weights = np.random.uniform(-0.1, 0.1, size=n_features)
m = np.zeros(n_features)
v = np.zeros(n_features)

# Обучение с Adam optimizer
for epoch in range(1, EPOCHS + 1):
    # Прямое распространение и вычисление ошибки
    errors = X_poly @ weights - y

    # Градиент
    gradient = X_poly.T @ errors / len(X)

    # Обновление моментов Adam
    m = BETA1 * m + (1 - BETA1) * gradient
    v = BETA2 * v + (1 - BETA2) * gradient**2

    # Коррекция смещения
    m_hat = m / (1 - BETA1**epoch)
    v_hat = v / (1 - BETA2**epoch)

    # Обновление весов
    weights -= LEARNING_RATE * m_hat / (np.sqrt(v_hat) + EPSILON)

    # Логирование
    if epoch % LOG_INTERVAL == 0:
        loss = np.mean(errors**2)
        print(f"Эпоха {epoch:4d}, Loss: {loss:.6f}")

# Вывод результатов обучения
print("\nОбученные веса:")
print(tabulate(zip(feature_names, weights),
              headers=["Признак", "Вес"],
              tablefmt="grid",
              floatfmt=".6f"))

# Проверка на обучающей выборке
train_results = np.column_stack([
    [f"d={x[0]:.1f}, h={x[1]:.1f}" for x in X],
    np.round(y, 2),
    np.round(X_poly @ weights, 2),
    np.round(np.abs(X_poly @ weights - y), 2)
])

print("\nПроверка на обучающих данных:")
print(tabulate(train_results,
              headers=["Параметры", "Истинный объем", "Предсказание", "Ошибка"],
              tablefmt="grid"))

# Функция для предсказания
def predict_volume(cylinders, weights):
    """Предсказывает объем для массива цилиндров (d, h)."""
    d = cylinders[:, 0]
    h = cylinders[:, 1]
    r = d/2
    features = np.column_stack([
        np.ones(len(cylinders)),
        d, h,
        np.pi * (r**2) * h,
        d*h,
        d**2, h**2,
        (d + h)**2,
        r**2 * h
    ])
    return features @ weights

# Тестирование на новых данных
test_cylinders = np.array([
    [3.0, 5.0],   # V = π*(1.5)²*5 ≈ 35.34
    [5.0, 2.0],   # V = π*(2.5)²*2 ≈ 39.27
    [8.0, 10.0]   # V = π*4²*10 ≈ 502.65
])

predictions = predict_volume(test_cylinders, weights)
true_volumes = np.pi * (test_cylinders[:,0]/2)**2 * test_cylinders[:,1]

print("\nТестирование на новых данных:")
test_results = np.column_stack([
    [f"d={c[0]}, h={c[1]}" for c in test_cylinders],
    np.round(true_volumes, 2),
    np.round(predictions, 2),
    np.round(np.abs(predictions - true_volumes), 2)
])

print(tabulate(test_results,
              headers=["Параметры", "Ожидаемый объем", "Предсказание", "Ошибка"],
              tablefmt="grid"))

Эпоха  500, Loss: 8.738824
Эпоха 1000, Loss: 1.260297
Эпоха 1500, Loss: 0.677618
Эпоха 2000, Loss: 0.383411
Эпоха 2500, Loss: 0.235417
Эпоха 3000, Loss: 0.142520
Эпоха 3500, Loss: 0.079210
Эпоха 4000, Loss: 0.040704
Эпоха 4500, Loss: 0.021105
Эпоха 5000, Loss: 0.012484

Обученные веса:
+-----------+-----------+
| Признак   |       Вес |
+===========+===========+
| 1         | -0.025178 |
+-----------+-----------+
| d         | -0.287922 |
+-----------+-----------+
| h         |  0.053416 |
+-----------+-----------+
| π(d/2)²h  |  0.767713 |
+-----------+-----------+
| d*h       |  0.028416 |
+-----------+-----------+
| d²        |  0.009672 |
+-----------+-----------+
| h²        | -0.038282 |
+-----------+-----------+
| (d+h)²    |  0.023502 |
+-----------+-----------+
| (d/2)²h   |  0.695191 |
+-----------+-----------+

Проверка на обучающих данных:
+--------------+------------------+----------------+----------+
| Параметры    |   Истинный объем |   Предсказание |   Ошибка |
+=======